In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

def scorer(estimator, X, y):
    return -log_loss(y, estimator.predict_proba(X))

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
df=pd.read_json('../input/train.json')
df['priceperbed']=(df['price'].clip(upper=7000)/df['bedrooms'].clip(lower=1))
df['created']=df['created'].astype(np.datetime64)
df['created_day']=np.array(df.created.values, dtype='datetime64[D]').astype(np.float32)%7
df['created_week']=np.array(df.created.values, dtype='datetime64[W]').astype(np.float32)
df['created_hour']=np.array(df.created.values, dtype='datetime64[h]').astype(np.float32)%24
df['desc_count']=df.description.apply(lambda x: len(x.split())).clip(upper=150)
df['features_count']=df.features.apply(lambda x: len(x))
df['photos_count']=df.photos.apply(lambda x: len(x))

df['features']=df['features'].apply(lambda x: list(map(str.lower, x)))
for feature in ['hardwood floors',  'laundry in building', 'cats_allowed']:
    df[feature]=df['features'].apply(lambda x: feature in x)

df['medium']=df['interest_level']=='medium'
df['low']=df['interest_level']=='low'
df['high']=df['interest_level']=='high'

In [ ]:
x=[]
y_train=[]
y_val=[]
for i in range(100, df_train.shape[0], 3000):
    x+=[i]
    clf=ExtraTreesClassifier(max_depth=21, n_estimators=100, random_state=0)
    clf.fit(df_train.iloc[:i][cols].values, df_train.iloc[:i]['interest_level'])
    y_pred=clf.predict_proba(df_train.iloc[:i][cols])
    score=log_loss(df_train.iloc[:i]['interest_level'].values, y_pred)
    y_train+=[score]
    y_pred=clf.predict_proba(df_test[cols])
    score2=log_loss(df_test['interest_level'].values, y_pred)
    y_val+=[score2]
    print("%.6f %.6f"%(score, score2))    

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

kf = KFold(n_splits=4)
score_sum=0
score2_sum=0
for ktrain, ktest in kf.split(df_train):
    clf=KNeighborsClassifier(n_neighbors=500)
    clf.fit(df_train.iloc[ktrain][cols].values, df_train.iloc[ktrain]['interest_level'])
    y_pred=clf.predict_proba(df_train.iloc[ktrain][cols])
    score=log_loss(df_train.iloc[ktrain]['interest_level'].values, y_pred)
    score_sum+=score
    y_pred=clf.predict_proba(df_train.iloc[ktest][cols])
    score2=log_loss(df_train.iloc[ktest]['interest_level'].values, y_pred)
    score2_sum+=score2
    print("%.6f %.6f"%(score, score2))
print("means")
print("%.6f %.6f"%(score_sum/4, score2_sum/4))

In [ ]:
features_rows=(df.features.values)
features_rows=np.hstack(features_rows)
df_features=pd.DataFrame(features_rows, columns=['feature'])
df_feature_counts=pd.DataFrame(df_features.groupby(['feature']).size(), 
                               columns=['size']).reset_index()
df_feature_counts=df_feature_counts.sort_values('size', ascending=False)
feature_list=df_feature_counts.loc[df_feature_counts['size']>5000]['feature'].values
feature_list

In [ ]:
df_feature_counts[(df_feature_counts.feature.str.find("laundry")!=-1)]

In [ ]:
feature='dining room'
df[feature]=df['features'].apply(lambda x: feature in x)
d={}
for j in ['high', 'medium', 'low']:
    d[j]=[]
    for i in [True, False]:
        summa=(df[feature]==i).sum()
        d[j]+=[((df[feature]==i)&(df.interest_level==j)).sum()/summa]
plt.bar([0, 1], d['low'])
plt.bar([0, 1], d['medium'], bottom=d['low'])
plt.bar([0, 1], d['high'], bottom=np.sum([d['low'], d['medium']], axis=0))
plt.show()        

In [ ]:
df['features']=df['features'].apply(lambda x: list(map(str.lower, x)))
df['cats_allowed']=df['features'].apply(lambda x: 'cats allowed' in x)

In [ ]:
cols=['price', 'bathrooms', 'bedrooms', 'priceperbed', 'created_hour', 
          'desc_count', 'photos_count', 'features_count', 'hardwood floors',
          'laundry in building', 'cats_allowed', 'created_day']
df_train, df_test = train_test_split(df, random_state=0)


In [ ]:
kf = KFold(n_splits=4)
score_sum=0
score2_sum=0
for ktrain, ktest in kf.split(df_train):
    clf=ExtraTreesClassifier(max_depth=13, n_estimators=100, min_samples_split=12, random_state=0)
    clf.fit(df_train.iloc[ktrain][cols].values, df_train.iloc[ktrain]['interest_level'])
    y_pred=clf.predict_proba(df_train.iloc[ktrain][cols])
    score=log_loss(df_train.iloc[ktrain]['interest_level'].values, y_pred)
    score_sum+=score
    y_pred=clf.predict_proba(df_train.iloc[ktest][cols])
    score2=log_loss(df_train.iloc[ktest]['interest_level'].values, y_pred)
    score2_sum+=score2
    print("%.6f %.6f"%(score, score2))
print("means")
print("%.6f %.6f"%(score_sum/4, score2_sum/4))

In [ ]:
    clf=ExtraTreesClassifier(max_depth=12, n_estimators=100, random_state=0)
                            #min_samples_split=12, 
    clf.fit(df_train[cols].values, df_train['interest_level'])
    y_pred=clf.predict_proba(df_train[cols])
    score=log_loss(df_train['interest_level'].values, y_pred)
    y_pred=clf.predict_proba(df_test[cols])
    score2=log_loss(df_test['interest_level'].values, y_pred)
    print("%.6f %.6f"%(score, score2))